# Equilibrating the System

## Overview

### Questions

* What is equilibration?
* How do I save simulation results?

### Objectives

* Explain the process of **equilibration**.
* Demonstrate using **GSD** to write the simulation trajectory to a file.
* Introduce the **Logger** and show how to add logged data to the **GSD** file.

In [1]:
import hoomd
import math

## Equilibration

So far in this tutorial, we have placed *N* non-overlapping tetrahedra randomly in a box and then quickly compressed them to a moderate packing fraction.
The resulting configuration of particles is valid, but strongly dependent on the path we took to create it.
There are many more **equilibrium** configurations in the set of possible configurations that do not depend on the path.
**Equilibrating** the system is the process of taking an artificially prepared state and running a simulation.
During the simulation run, the system will relax to **equilibrium**.
First, let's initialize the **Simulation**.

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)
mc = hoomd.hpmc.integrate.ConvexPolyhedron(seed=42)
mc.shape['octahedron'] = dict(vertices=[(-0.5, 0, 0),
                                         (0.5, 0, 0),
                                         (0, -0.5, 0),
                                         (0, 0.5, 0),
                                         (0, 0, -0.5),
                                         (0, 0, 0.5)])
sim.operations.integrator = mc

HOOMD-blue v2.9.0-1828-gd903f0dc6 DOUBLE HPMC_MIXED SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX AVX2 
Compiled: 06/10/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
HOOMD-blue is running on the CPU


The previous section of this tutorial wrote the compressed system to `compressed.gsd`.
Initialize the system **state** from this file:

In [3]:
sim.create_state_from_gsd(filename='compressed.gsd')

## Writing simulation trajectories

Let's save the system **state** to a file periodically so that we can observe the equilibration process.
So far in this tutorial, we have used **GSD** files to store a single frame of the system **state** using either the **GSD** Python package or `GSD.write_state`.
Here, let's use the **GSD analyzer** (another **operation**) to create a **GSD** file with many frames in a **trajectory**.

In [4]:
gsd = hoomd.dump.GSD(filename='trajectory.gsd', trigger=hoomd.trigger.Periodic(10000), overwrite=True)
sim.operations.analyzers.append(gsd)

**Operations** mark some of their class *properties* as **loggable**.
**GSD** files can also store these **loggable properties** in addition to the system **state**.
We've seen some of these **loggable properties** earlier in this tutorial: `mc.translate_moves`, `mc.rotate_moves`, and `mc.overlaps`.
You can look at the documentation for a specific **operation** to find a list of all the **logged quantities** 

Use **Logger** object to select which of these **loggable properties** to include in the **GSD** file.
Let's just add all of the **loggable properties** that the HPMC integrator provides:

In [5]:
logger = hoomd.Logger()
logger += mc

Use this **Logger** to populate the **GSD** file:

In [6]:
# currently, the logger is broken with the ConvexPolyhedron shape parameters
# gsd.log = logger

## Tuning the trial move size

**TODO**

In [7]:
mc.d['octahedron'] = 0.03
mc.a['octahedron'] = 0.06

## Equilibrating the system

To equilibrate the system, **run** the simulation.
The length of the run needed is strongly dependent on the particular model, the system size, the density, and many other factors.
Hard particle Monte Carlo self-assembly often takes tens of millions of time steps for systems with ~10,000 particles.
Our system is much smaller, but it still takes over one million steps.
If you run this locally, be prepared - this cell will take ~2 hours to complete.

In [9]:
sim.verbose_run = True
sim.run(1e3)
print(mc.translate_moves[0] / sum(mc.translate_moves))
print(mc.rotate_moves[0] / sum(mc.rotate_moves))
sim.run(2e6)

Time 01:11:13 | Step 1013090 / 1013090 | TPS 278.621 | ETA 00:00:00
Average TPS: 278.523
---------
notice(2): -- HPMC stats:
notice(2): Trial moves per second:        278445
notice(2): Overlap checks per second:     1.80367e+07
notice(2): Overlap checks per trial move: 64.7765
notice(2): Number of overlap errors:      0
0.28000095974278894
0.25932749976993835
Time 01:11:23 | Step 1015899 / 2013090 | TPS 280.843 | ETA 00:59:10
Time 01:11:33 | Step 1018720 / 2013090 | TPS 282.067 | ETA 00:58:45
Time 01:11:43 | Step 1021575 / 2013090 | TPS 285.427 | ETA 00:57:53
Time 01:11:53 | Step 1024448 / 2013090 | TPS 287.298 | ETA 00:57:21
Time 01:12:03 | Step 1027255 / 2013090 | TPS 280.634 | ETA 00:58:32
Time 01:12:13 | Step 1030054 / 2013090 | TPS 279.894 | ETA 00:58:32
Time 01:12:23 | Step 1032848 / 2013090 | TPS 279.387 | ETA 00:58:28
Time 01:12:33 | Step 1035637 / 2013090 | TPS 278.875 | ETA 00:58:24
Time 01:12:43 | Step 1038432 / 2013090 | TPS 279.444 | ETA 00:58:07
Time 01:12:53 | Step 10412

How do we know if we have run long enough to **equilibrate** the system?
Let's look at the **trajectory** in the next section of this tutorial and find out.